In [1]:
import torch, torchvision
import numpy as np
import pandas as pd
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import math

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


# MLP

In [11]:
# Getting the dataset
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False)

X = trainset.data
y = trainset.targets
X = X.type(torch.float)
X = (X/255)

X_test = testset.data
y_test = testset.targets
X_test = X_test.type(torch.float)
X_test = (X_test/255)


# Preprocessing/Transforming
# Define a series of transformations
transform = transforms.Compose([
    #transforms.RandomRotation(20),
    transforms.Lambda(lambda x: x.view(x.size(0), -1)) # Flatten the tensor
])

X_t = transform(X)
y_t = y


X_test = transform(X_test)
X_test_gpu = X_test.to(device)
y_test_gpu = y_test.to(device)


X_gpu = X_t.to(device)
y_gpu = y.to(device)
# Convert to Dataset and put in a DataLoader
tensor_dataset = TensorDataset(X_gpu, y_gpu)
dataloader = DataLoader(tensor_dataset,batch_size=128,shuffle = True)


tensor_dataset_test = TensorDataset(X_test, y_test)
dataloader_test = DataLoader(tensor_dataset_test,batch_size=1)


In [12]:
# Custom Linear Layer
class CustomLinearLayer(torch.nn.Module):
    def __init__(self, input_size, output_size,bias=True):
        super(CustomLinearLayer, self).__init__()

        self.weight = nn.Parameter(torch.Tensor(output_size,input_size))
        if bias:
          self.bias = nn.Parameter(torch.Tensor(output_size))

        #nn.init.kaiming_uniform_(self.weight.data, a=0, mode='fan_in', nonlinearity='relu') # kaiming He weight initialization
        #nn.init.kaiming_uniform_(self.bias.data, a=0, mode='fan_in', nonlinearity='relu')

        # Define parameters (weights and bias)
        if True:
          stdv = 1. / math.sqrt(self.weight.data.size(1))
          self.weight.data.uniform_(-stdv, stdv)
          if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        # Implement the forward pass
        #print(x.shape)
        #x = torch.matmul(x, self.weight.T)
        x = x@self.weight.T + self.bias
        return x

In [ ]:
class model_linear(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main = torch.nn.Sequential(
            torch.nn.Linear(784, 3000,bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(3000, 3000,bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(3000, 1000,bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(1000, 500,bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(500,10,bias=True),
            torch.nn.Softmax(dim =1)
        )

    def forward(self,X):
        y_pred = self.main(X)
        return y_pred

    def predict(self,X):
        with torch.no_grad():
            y_pred = self.main(X)
            y_class = torch.argmax(y_pred,dim=1)
        return y_class

In [13]:
class model_custom_linear(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main = torch.nn.Sequential(
            CustomLinearLayer(784, 3000,bias=True),
            torch.nn.ReLU(),
            CustomLinearLayer(3000, 3000,bias=True),
            torch.nn.ReLU(),
            CustomLinearLayer(3000, 1000,bias=True),
            torch.nn.ReLU(),
            CustomLinearLayer(1000, 500,bias=True),
            torch.nn.ReLU(),
            CustomLinearLayer(500,10,bias=True),
            torch.nn.Softmax(dim =1)
        )

    def forward(self,X):
        y_pred = self.main(X)
        return y_pred

    def predict(self,X):
        with torch.no_grad():
            y_pred = self.main(X)
            y_class = torch.argmax(y_pred,dim=1)
        return y_class

In [17]:
# Instantiate your model and transfer it to GPU (if available)
#model = model_linear().to(device)
model = model_custom_linear().to(device)

In [18]:
opt = torch.optim.Adadelta(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

train_acc_list = []
test_acc_list = []

for epoch in range(5):
    for batch, (X,y) in enumerate(dataloader):
        #print(f'Batch {batch}, Loss {loss}')
        opt.zero_grad()
        y_pred = model.forward(X)
        loss = criterion(y_pred,y)
        loss.backward()
        for param in model.parameters():
          #print('\nGradient',param.grad.data)
          1+1

        opt.step()

        if ((batch%500) == 0) or False:
            acc_batch_train = sum((torch.argmax(y_pred,dim=1)-y) == 0)/len(y)
            y_test_pred = model.forward(X_test_gpu[:1000])

            acc_test = sum((torch.argmax(y_test_pred,dim=1)-y_test_gpu[:1000]) == 0)/1000

            print('Batch:',batch,' Training Accuracy:', acc_batch_train.item())
            print('Testing Accuracy:',acc_test.item())
            train_acc_list.append(acc_batch_train.item())
            test_acc_list.append(acc_test.item())

    print(f'\n\nEpoch {epoch} \n\n')



Batch: 0  Training Accuracy: 0.1015625
Testing Accuracy: 0.125


Epoch 0 


Batch: 0  Training Accuracy: 0.8671875
Testing Accuracy: 0.8910000324249268


Epoch 1 


Batch: 0  Training Accuracy: 0.9140625
Testing Accuracy: 0.9330000281333923


Epoch 2 


Batch: 0  Training Accuracy: 0.9765625
Testing Accuracy: 0.9660000205039978


Epoch 3 


Batch: 0  Training Accuracy: 0.9609375
Testing Accuracy: 0.9720000624656677


Epoch 4 




In [ ]:
y_test_pred = model.forward(X_test_gpu)
acc_test = sum((torch.argmax(y_test_pred,dim=1)-y_test_gpu) == 0)/10000
print('Test accuracy is:',acc_test)


Test accuracy is: tensor(0.9681, device='cuda:0')


# CNN

In [ ]:
# Getting the dataset
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False)

X = trainset.data
y = trainset.targets
X = X.type(torch.float)
X = (X/255)

X_test = testset.data
y_test = testset.targets
X_test = X_test.type(torch.float)
X_test = (X_test/255)


# Preprocessing/Transforming
# Define a series of transformations
transform = transforms.Compose([
    #transforms.RandomRotation(20),
    #transforms.Lambda(lambda x: x.view(x.size(0), -1)) # Flatten the tensor
])

X_t = transform(X)
y_t = y


X_test = transform(X_test)
X_test_gpu = X_test.to(device)
y_test_gpu = y_test.to(device)


X_gpu = X_t.to(device)
y_gpu = y.to(device)
# Convert to Dataset and put in a DataLoader
tensor_dataset = TensorDataset(X_gpu, y_gpu)
dataloader = DataLoader(tensor_dataset,batch_size=100,shuffle = True)


tensor_dataset_test = TensorDataset(X_test, y_test)
dataloader_test = DataLoader(tensor_dataset_test,batch_size=100)


In [ ]:
# Convolutional Neural Network

class Conv_Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 1,out_channels = 6,kernel_size = (3,3),stride = 1,padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
            torch.nn.Conv2d(in_channels = 6,out_channels = 16,kernel_size = (5,5),stride = 1,padding = 0),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size = 2,stride = 2),
            torch.nn.Flatten(start_dim = 1),
            torch.nn.Linear(49*16,120),
            torch.nn.ReLU(),
            torch.nn.Linear(120,84),
            torch.nn.ReLU(),
            torch.nn.Linear(84,10),
            torch.nn.Softmax(dim = 1)
        )


    def forward(self,X):
        y_pred = self.main(X)
        return y_pred

In [ ]:
# Instantiate your model and transfer it to GPU (if available)
model = Conv_Network().to(device)


In [ ]:
opt = torch.optim.Adadelta(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

train_acc_list = []
test_acc_list = []

for epoch in range(5):
    for batch, (X,y) in enumerate(dataloader):
        #print(f'Batch {batch}, Loss {loss}')
        opt.zero_grad()
        y_pred = model.forward(X)
        loss = criterion(y_pred,y)
        loss.backward()
        opt.step()

        if ((batch%50) == 0) or True:
            acc_batch_train = sum((torch.argmax(y_pred,dim=1)-y) == 0)/len(y)
            y_test_pred = model.forward(X_test_gpu[:1000])

            acc_test = sum((torch.argmax(y_test_pred,dim=1)-y_test_gpu[:1000]) == 0)/1000

            print('Batch:',batch,' Training Accuracy:', acc_batch_train.item())
            print('Testing Accuracy:',acc_test.item())
            train_acc_list.append(acc_batch_train.item())
            test_acc_list.append(acc_test.item())

    print(f'\n\nEpoch {epoch} \n\n')



RuntimeError: Given groups=1, weight of size [6, 1, 3, 3], expected input[1, 100, 28, 28] to have 1 channels, but got 100 channels instead

In [ ]:
%matplotlib inline
img = dataloader.dataset.tensors[0][2].view(28,28)
plt.imshow(img, cmap='gray')  # Use the 'gray' colormap for grayscale images
plt.show()